In [2]:
import numpy as np
import pandas as pd
import json
import copy

import bqplot as bqp
import seaborn as sns
# import bqviz as bqv
from ipyleaflet import *
from ipywidgets import *
from IPython.display import display

In [5]:
with open('china2.json', encoding='utf-8') as f:
    data = json.load(f)
    
port_coordinates = pd.read_csv('port_lat_long.csv')

df_meta = pd.read_excel('Mysteel_tickers.xlsx').set_index('Ticker')

data['features'][0]['properties']

{'childNum': 18,
 'cp': [84.9023, 42.148],
 'id': 65,
 'name': 'Xinjiang',
 'size': '550'}

In [18]:
m = Map(center=[34.14, 120.23], zoom=4, name='Base map', 
        basemaps=basemaps.OpenStreetMap.DE,
       layout=Layout(width='300px', height='400px'))


polygon_default_layout = {
    'color': 'grey',
    'weight':1,
    'fillColor':'grey',
    'fillOpacity':0.2
}

polygon_highlighted_layout = {
    'color': 'grey',
    'weight':1,
    'fillColor':'green',
    'fillOpacity':0.2
}


label = Label(value = 'test', layout = Layout(width='20%'))

for feature in data['features']:
    feature['properties']['style'] = polygon_default_layout
    
geojson_layer = GeoJSON(data = data, name = 'China Provinces', hover_style={'fillColor':'green'})


selected_set = set()
selected_layer = None

def convert_selected_set_to_geojson(selected_set):
    geojson ={'type': 'FeatureCollection', 'features': []}
    geojson['features'] = [feature for feature in data['features'] if feature['properties']['name'] in selected_set]
    for feature in data['features']:
        feature['properties']['style'] = polygon_highlighted_layout
    return geojson

def selected_on_click_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    cid = properties['name']
    selected_set.remove(cid)
    if selected_layer is not None:
        m.remove_layer(selected_layer)
    selected_layer = GeoJSON(data = convert_selected_set_to_geojson(selected_set),
                            name = 'Selected provinces',
                            hover_style = {'fillcolor': 'green'})
    selected_layer.on_click(selected_on_click_handler)
    m.add_layer(selected_layer)
        
def geojson_onclick_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    cid = properties['name']
    selected_set.add(cid)
    if selected_layer is not None:
        m.remove_layer(selected_layer)
    selected_layer = GeoJSON(data = convert_selected_set_to_geojson(selected_set),
                    name = 'Selected provinces',
                    hover_style = {'fillcolor':'green'})
    selected_layer.on_click(selected_on_click_handler)
    m.add_layer(selected_layer)
    

geojson_layer.on_click(geojson_onclick_handler)
m.add_layer(geojson_layer)
    
    
m

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…